# # Optimiser module
# Run all cells
# Then return to deep learning module at
# ./neuralnet/helpers/GRU.ipynb

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [2]:
using DelimitedFiles

function initialiseData(path, startindex, finalindex)
    full_data = readdlm(path,',')
    full_data = round.(Int, full_data)
    full_data = full_data[2:end, :]
    data = full_data[:, startindex:finalindex]
    return full_data, data
end


initialiseData (generic function with 1 method)

# # Convert piano roll into timestepx12 tonal mapping


In [3]:
function getActiveNotes(data, timestep)
    return findall(x->x==1, data[:,timestep])
end

function createTonalMap(full_data)
    tonal_map = zeros(Int64, size(full_data, 2), 12)

    for i in 1:size(full_data, 2)
        active_notes = getActiveNotes(full_data, i)
        for note in active_notes
            # due to 1-indexing, real note is one step below
            note = note-1
            tonal_map[i, (note%12)+1] += 1
        end
    end

    return tonal_map
end


createTonalMap (generic function with 1 method)

# # Pattern matcher to identify the best-fit key and scale from existing notes

In [4]:
#Score In: Tx12 Matrix, pattern. Out: Best position(s)
function score_key(matrix, pattern)
    counter = zeros(Int64, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    println("Key Counter is ", counter)
    
    key_score = zeros(Int64, 12, 1)
    
    for i in 1:size(pattern, 1)
        for j in 1:size(pattern, 1)
            key_score[i] += circshift(pattern, i-1)[j]*counter[j]
        end
    end
    println("Key Score is ", key_score)
    return key_score
end

# Find maximum from list
function findmaximum(list)
    positions = []
    max_value = maximum(list)
    for (i, x) in enumerate(list)
        if x == max_value
            push!(positions, i)
        end
    end
    return positions
end

# Find an optimal scale
function score_tonality(matrix, positions)
    
    majorpattern = [1, 0, 0.25, 0, 0.25, 1, 0, 1, 0, 0.25, 0, 1]
    minorpattern = [0.25, 0, 1, 0, 1, 0.25, 0, 0.25, 0.75, 1, 0, 0.25]
    
    counter = zeros(Int64, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    
    tonality_scores = []
    
    for position in positions
        counter_shifted = circshift(counter, (0,-(position-1)))
        println("Position Counter for ", position, " is ", counter_shifted)
        
        major_score = 0
        minor_score = 0
        
        for i in 1:size(counter_shifted, 2)
            major_score += majorpattern[i]*counter_shifted[i]
            minor_score += minorpattern[i]*counter_shifted[i]
        end

        if major_score > minor_score
            push!(tonality_scores, [position, major_score, 1])
        else
            push!(tonality_scores, [position, minor_score, 0])
        end
        
    end 
    
    return tonality_scores
end

score_tonality (generic function with 1 method)

In [5]:
function findScale(full_data, tonal_map)
    # Scale patterns
    pattern = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]
    Keys = Dict{Integer, String}(1 => "C", 2 => "C#", 3 => "D", 4 => "D#", 5 => "E", 6 => "F", 7 => "F#", 8 => "G", 9 => "G#", 10 => "A", 11 => "A#", 12 => "B")

    # Score matrix after pattern
    key_score_result = score_key(tonal_map, pattern)
    # score_result = [9, 5, 18, 9, 23, 18, 8, 2, 7, 16, 12, 11]

    # Find positions of maximum score
    positions = findmaximum(key_score_result)
    sort!(positions)

    # # Use positions to find the best key

    tonality_results = score_tonality(tonal_map, positions)
    println(tonality_results)

    # [Position (1 = C), Score, Scale (1 = Major, 0 = Minor)]
    best_key = [0, 0, 1]

    # In case of tied scores, check each for the best scale
    for result_group in tonality_results
        if best_key[2] < result_group[2]
            best_key[1] = result_group[1]
            best_key[2] = round.(Int, result_group[2])
            best_key[3] = result_group[3]
        end
    end

    #Save Key
    println(best_key)
    if best_key[3] == 1
        key = best_key[1]
        key_print = Keys[best_key[1]]
        scale = "Major"
    else
        key = ( (12+(best_key[1] - 1) - 3)%12 ) + 1
        key_print = Keys[key]
        scale = "Minor"
    end
    println(key, " ", key_print, " ", scale)
    return key, scale
end
    
#key, scale = findScale(tempfull, tonalmap)
#println(key, " ", scale)

findScale (generic function with 1 method)

# # Define patterns for chord functions relative to key

In [6]:
function initialiseFunctionDefinitions(key, scale)
    # Function entry: "Name" => ([notes], [Bass note], [Double/Triple notes], [Rhythmical Weight (Strong, Weak, Off)])
    functions = Dict{String, Array{Int64, 1}}

    if scale == "Major"
    #Major  =  [1, 0, 2, 0, 3, 4, 0, 5, 0, 6, 0, 7]
    #

        functions = Dict(
        "T"    =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O", "A", "C"]),
        "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O", "A", "C"]),
        "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "A", "C"]),
        "T-5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O", "C"]),
        "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "C"]),
        "T/3"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((4+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["S", "W", "O", "C"]),
        "S/3"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["S", "W", "O", "C"]),
        "D/3"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "C"]),
        "T/5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W", "O"]),
        "S/5"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W", "O"]),
        "D/5"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W", "O"]),
        "D64"  =>  (circshift([4, 0, 0, 0, 6, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "A", "C"]),
        "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 0, 6, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S", "O"]),
        "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S", "O"]),
        "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "A", "C"]),
        "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O", "A", "C"]),
        "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ nothing ], ["S", "W", "O", "A", "C"]),
        "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O", "A", "C"]),
        "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["S", "W", "O", "A", "C"]),
        "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["S", "W", "O", "A", "C"]),
        "D7-5" =>  (circshift([0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "A", "C"]),
        "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
        "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S"]),
        "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "Ss"   =>  (circshift([0, 0, 1, 0, 0, 3, 0, 0, 0, 5, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 1) ], [ (((2+(key-1))%12)+1, 1), (((5+(key-1))%12)+1, 3) ], ["S", "W", "O"]),
        "Ts"   =>  (circshift([3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 1) ], [ (key, 3), (((9+(key-1))%12)+1, 1) ], ["S", "W", "O", "C"]),
        "Tm"   =>  (circshift([0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 5], (key-1)), [ (((4+(key-1))%12)+1, 1) ], [ (((4+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["S", "W", "O"]))

    elseif scale == "Minor"
    #                         [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11]
    #Minor  =                 [1, 0, 2, 3, 0, 4, 0, 5, 6, 0, 7, 7+]              
        functions = Dict(
        "T"    =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O", "A", "C"]),
        "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O", "A", "C"]),
        "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "A", "C"]),
        "T-5"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O", "C"]),
        "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "C"]),
        "T/3"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((3+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["S", "W", "O", "C"]),
        "S/3"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((8+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["S", "W", "O", "C"]),
        "D/3"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "C"]),
        "T/5"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W", "O"]),
        "S/5"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W", "O"]),
        "D/5"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W", "O"]),
        "D64"  =>  (circshift([4, 0, 0, 6, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "A", "C"]),
        "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 6, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
        "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
        "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "A", "C"]),
        "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O", "A", "C"]),
        "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ nothing ], ["S", "W", "O", "A", "C"]),
        "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O", "A", "C"]),
        "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["S", "W", "O", "A", "C"]),
        "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["S", "W", "O", "A", "C"]),
        "D7-5" =>  (circshift([0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "A", "C"]),
        "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
        "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S"]),
        "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
        "Dm"   =>  (circshift([0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 1, 0], (key-1)), [ (((10+(key-1))%12)+1, 1) ], [ (((10+(key-1))%12)+1, 1), (((2+(key-1))%12)+1, 3) ], ["S", "W", "O"]),
        "Ts"   =>  (circshift([3, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0], (key-1)), [ (((8+(key-1))%12)+1, 1) ], [ (key, 3), (((8+(key-1))%12)+1, 1) ], ["S", "W", "O", "C"]),
        "Tm"   =>  (circshift([0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 5, 0], (key-1)), [ (((3+(key-1))%12)+1, 1) ], [ (((3+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["S", "W", "O"]))
    end
        
    return functions
end
    
#functions = initialiseFunctionDefinitions()

initialiseFunctionDefinitions (generic function with 1 method)

# # Analyze tonal map and calculate tonal distance from original data for each timestep

In [7]:
function tonal_distance(input, target, double_notes, triple=false, debug=false)
    input = deepcopy(input)
    target = deepcopy(target)
    target = map(x -> x > 0 ? 1 : 0, target)

    double_index = []
    two_doubles = false
            
    if !isnothing(double_notes[1])
        if size(double_notes, 1) == 2
            two_doubles = true
            double_index = [double_notes[1][1], double_notes[2][1]]
        end
        for note in double_notes
            if triple
                target[note[1]] += 2
            else
            target[note[1]] += 1
            end
        end
    end
    
    if debug
        println("Input:  ", input)
        println("Target: ", target)
    end
          
    
    distance = 0

    
    subvector = target - input
    #println(subvector)
    
    # Subtract input-vector from target vector. Any notes that are in place are removed immediately. The rest are resolved in a left/right iterative "adjacency search", counting the distance
    # moved until opposite signed note is found for resolution. NB: chords with multiple possible doubled notes are resolved to the closest match, distancewise.
    k = 1
    j = 1

    while any(x->x != 0, subvector)

        i = 6+k
        paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])
        if debug
            println(paddedsub)
            println(i)
        end

        if paddedsub[i] != 0 
            if sign(paddedsub[i-j]) != sign(paddedsub[i]) && paddedsub[i-j] != 0
                
                regular = (i-j+6) % 12
                if regular == 0
                    regular = 12
                end
                if debug
                    println("k: ", k)
                    println("i: ", i)
                    println("i-j: ", i-j)
                    println("regular: ", regular)
                end

                if two_doubles && (regular == double_index[1] || regular == double_index[2])

                    
                    for index in double_index
                        subvector[index] -= 1
                    end
                    subvector[k] += 1
                    distance += j
                    
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])

                    two_doubles = false

                    if debug
                        println("Double in i-j+6")
                        println(subvector)
                    end
                elseif two_doubles && (k == double_index[1] || k == double_index[2])

                    
                    for index in double_index
                        subvector[index] -= 1
                    end
                    subvector[regular] += 1
                    distance += j

                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])
                    two_doubles = false

                    if debug
                        println("Double in k for i-j+6")
                        println(subvector)
                    end
                else
                    
                    num = subvector[regular]
                    subvector[regular] = subvector[regular] + (subvector[k] / abs(subvector[k]))
                    subvector[k] = subvector[k] + (num / abs(num))
                    distance += j
                    
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])

                    if debug
                        println("Normal found in i-j")
                        println(subvector)
                    end     
                end

            elseif sign(paddedsub[i+j]) != sign(paddedsub[i]) && paddedsub[i+j] != 0
                regular = (i+j-6) % 12
                if regular == 0
                    regular = 12
                end
                            
                if debug
                    println("k: ", k)
                    println("i: ", i)
                    println("i+j: ", i+j)
                    println("regular: ", regular)
                end

                if two_doubles && (regular == double_index[1] || regular == double_index[2])
                    
                    for index in double_index
                        subvector[index] -= 1
                    end
                    subvector[k] += 1
                    distance += j
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])

                    two_doubles = false

                    if debug
                        println("Double in i+j-6")
                        println(subvector)
                    end

                elseif two_doubles && (k == double_index[1] || k == double_index[2])

                    
                    for index in double_index
                        subvector[index] -= 1
                    end
                    subvector[regular] += 1
                    distance += j
                                    
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])
                    two_doubles = false

                    if debug
                        println("Double in k for i+j-6")
                        println(subvector)
                    end
                else

                    num = subvector[regular]
                    subvector[regular] = subvector[regular] + (subvector[k] / abs(subvector[k]))
                    subvector[k] = subvector[k] + num / abs(num)  
                    distance += j
                    paddedsub = vcat(subvector[7:12], subvector, subvector[1:6])

                    if debug
                        println("Normal found in i+j")
                        println(subvector)
                    end    
                end
            else
                if debug
                    println("No match found")
                end
                k += 1
            end
        else
            if debug
                println("Initial not found")
            end
            k += 1
        end
        if k == 13
            if debug 
                println("Iteration done for j ", j)
            end
            k = 1
            j += 1
        end
        if j == 7
            if debug
                println("Done")
            end
            break                    
        end
        if debug
            println(paddedsub)
        end
    end
    if debug
        println(subvector)
        println("Distance ", distance)
    end
    return distance
end
               
# Creates sorted and unsorted lists of the tonal distances of each time-step in startindex:finalindex in relation to all available functions
function initialiseFunctionPotentialList(startindex, finalindex, tonal_map, functions)

    # Used for searching best-first
    function_potential = Array{Tuple{Int64,String},1}[] 

    # Used for building shortest path map                            
    function_potential_unsorted = Array{Tuple{Int64,String},1}[] 

    for i in startindex:finalindex
        function_pool = Tuple{Int64, String}[]
        for chord in keys(functions)
            triple=false
            if chord == "T-5" || chord == "S-5" || chord == "D-5"                                                            
                triple = true
            end
            tonal_dist = 3*tonal_distance(tonal_map[i,:], functions[chord][1], functions[chord][3], triple)
            function_pool_entry = (tonal_dist, chord)        
            push!(function_pool, (tonal_dist, chord))
        end 
        push!(function_potential_unsorted, function_pool)
        push!(function_potential, sort(function_pool))
    end
    return function_potential, function_potential_unsorted
end
       

initialiseFunctionPotentialList (generic function with 1 method)

# # Construct the search space for note distribution to voices

In [8]:
using Combinatorics

# Finds and returns all possible distributions of the valid note classes for a given function

function getVoicingPermutations(functions, func)
    voicing_permutations = Array{Tuple{Int64,Int64}, 1}[]
    voicings_initial = []
    
    function_map = functions[func][1]
    double_note = functions[func][3]
    bass_note = functions[func][2][1]

    active_notes = sort!([(i, x) for (i, x) in enumerate(function_map) if x > 0], by = x -> x[2])   # [position, value]
    
    filter!(e->e != bass_note, active_notes)
        
    voicing_sans_bass = copy(active_notes)
            
    if size(active_notes, 1) == 1
        push!(voicing_sans_bass, bass_note)
        push!(voicing_sans_bass, bass_note)
        push!(voicings_initial, voicing_sans_bass)
    elseif size(active_notes, 1) == 2
        for i in 1:size(double_note, 1)
            voicing_sans_bass = copy(active_notes)
            push!(voicing_sans_bass, double_note[i])
            push!(voicings_initial, voicing_sans_bass)
        end
    elseif size(active_notes, 1) == 3
        push!(voicings_initial, voicing_sans_bass)
    end
                    
    for entry in voicings_initial
        voicings = collect(permutations(entry))
        
        for j in voicings
            push!(voicing_permutations, j)
        end
    end
    
    voicing_permutations = unique(voicing_permutations)   
    for p in voicing_permutations
        pushfirst!(p, bass_note)
    end
    
    return voicing_permutations
end
        
# Generates the candidate solutions of note distribution for a given function
               
function generateCandidates(functions, func)
    voicing_permutations = getVoicingPermutations(functions, func)
    valid_voicings = Array{Tuple{Int64,Int64},1}[]
    note_repository = []   # [[all C's], [all C#'s], [all D's], ..., [all B's]]
    for note_class in 1:12
        push!(note_repository, [x for x in 40:80 if x%12 == (note_class-1)])
    end

    
    
    for permutation in voicing_permutations
        #println(permutation)
        voicing = []
        for b in note_repository[permutation[1][1]]
            if b in 40:60
                for t in note_repository[permutation[2][1]]
                    if t in 47:68 && t >= b && t-b <= 20
                        for a in note_repository[permutation[3][1]]
                            if a in 52:72 && a >= t && a-t <= 12                                 
                                for s in note_repository[permutation[4][1]]
                                    if s in 59:80 && s >= a && s-a <= 12  
                                        voicing = Tuple{Int64,Int64}[]
                                        push!(voicing, (b, permutation[1][2]), (t, permutation[2][2]), (a, permutation[3][2]), (s, permutation[4][2])) 
                                        push!(valid_voicings, voicing)
                                    end                                
                                end           
                            end    
                        end
                        
                    end
                end       
            end
        end
    end
    
    ## Enforce Special rules
    
    if (func == "T-5") || (func == "D-5")
        placeholder = Array{Tuple{Int64,Int64},1}[]
        for voicing in valid_voicings
            if (voicing[4][2] == 1) && (voicing[4][1] - voicing[3][1] == 9 || voicing[4][1] - voicing[3][1] == 8)
                push!(placeholder, voicing)
            end
        end
        valid_voicings = placeholder
    end
    return(valid_voicings)
end
        
# Initialises all possible permutations of every function with regards to key and scale
function createVoicingPermutations(functions)

    voicing_permutations = Dict{String, Array{Array{Tuple{Int8,Int8},1}}}()
    voicing_permutations_raw = Array{Tuple{Int64,Int64},1}[]

    for keys in keys(functions)
        entry = unique(generateCandidates(functions, keys))
        #println(entry)
        #println(typeof(entry))
        voicing_permutations[keys] = entry
        for voicing in entry
            push!(voicing_permutations_raw, voicing)
        end
    end
    
    return voicing_permutations, voicing_permutations_raw
end


createVoicingPermutations (generic function with 1 method)

In [9]:
function createValidFunctionTable(scale)
    if scale == "Major"
        valid_function_table = Dict(
        "T"    => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Ss"],
        "S"    => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "T64", "T54", "D64", "D54", "Ts", "Tm", "Ss"],
        "D"    => ["T", "T-5", "T/3", "T/5", "S/3", "D", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ts"],
        "T-5"  => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Ss"],
        "D-5"  => ["T", "T-5", "T/3", "T/5", "S/3", "D", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ts"],
        "T/3"  => ["T", "S", "S/3", "S6", "S65", "S65/6", "D", "D/3", "D/5", "D7-5", "D7/5", "D64", "D54", "Tm", "Ts"],
        "S/3"  => ["T", "T/3", "T/5", "S", "S6", "D", "D/3", "D7", "D7/3", "T64", "T54", "D64", "D54", "Ts"],
        "D/3"  => ["T", "T-5", "S/3", "D", "D7", "D7-5", "Ts"],
        "T/5"  => ["S", "S/3", "S6", "S65", "S65/6", "D"],
        "S/5"  => ["T", "S6", "S65", "S65/6"],
        "D/5"  => ["T", "T/3"],
        "D64"  => ["D", "D7-5", "D54", "D64"],
        "T64"  => ["T", "T54", "T64"],
        "T54"  => ["T", "T54"],
        "D54"  => ["D", "D54"],
        "D7"   => ["T", "T-5", "D7", "D7/3", "D7/5", "D7/7", "Ts", "T64", "T54"],
        "D7/3" => ["T", "T-5", "D7", "D7/3", "D7/5", "D7/7", "Ts"],
        "D7/5" => ["T", "D7", "D7/3", "D7/5", "D7/7"],
        "D7/7" => ["T/3", "D7", "D7/3", "D7/5", "D7/7"],
        "D7-1" => ["T", "T/3", "D7", "D7-1", "D7/3", "D7/5", "D7/7", "Ts"],
        "D7-5" => ["T", "D7-5"],
        "S65"  => ["T/5", "D", "D64", "S65"],
        "S65/6"=> ["D", "D7", "D7-1", "D7-5", "T/5", "S65/6"],
        "S6"   => ["T/3", "D", "D7", "D7-5", "D64", "S6"],
        "Ts"   => ["T/3", "S", "S/3", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D7", "D7-1", "D7/3", "Ts", "Ss", "Tm"],
        "Tm"   => ["S", "D/3", "D7/3", "Ts", "Tm"],
        "Ss"   => ["T", "T-5", "T/3", "T/5", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ss"])
        # ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "T64", "T54", "D64", "D54", "Ts", "Tm", "Ss"],
    elseif scale == "Minor"
        valid_function_table = Dict(
        "T"    => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Dm"],
        "S"    => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "T64", "T54", "D64", "D54", "Ts", "Tm", "Dm"],
        "D"    => ["T", "T-5", "T/3", "T/5", "S/3", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ts"],
        "T-5"  => ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54", "Ts", "Tm", "Dm"],
        "D-5"  => ["T", "T-5", "T/3", "T/5", "S/3", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "Ts"],
        "T/3"  => ["T", "S", "S/3", "S6", "S65", "S65/6", "D", "D/3", "D/5", "D7-5", "D7/5", "D64", "D54", "Tm", "Ts"],
        "S/3"  => ["T", "T/3", "T/5", "S", "S6", "D", "D/3", "D7", "D7/3", "T64", "T54", "D64", "D54", "Ts"],
        "D/3"  => ["T", "T-5", "S/3", "D", "D7", "D7-5"],
        "T/5"  => ["S", "S/3", "S6", "S65", "S65/6", "D"],
        "S/5"  => ["T", "S6", "S65", "S65/6"],
        "D/5"  => ["T", "T/3"],
        "D64"  => ["D", "D7-5", "D54", "D64"],
        "T64"  => ["T", "T54", "T64"],
        "T54"  => ["T", "T54"],
        "D54"  => ["D", "D54"],
        "D7"   => ["T", "T-5", "D7", "D7/3", "D7/5", "D7/7", "T64", "T54", "Ts"],
        "D7/3" => ["T", "T-5", "D7", "D7/3", "D7/5", "D7/7"],
        "D7/5" => ["T", "D7", "D7/3", "D7/5", "D7/7"],
        "D7/7" => ["T/3", "D7", "D7/3", "D7/5", "D7/7"],
        "D7-1" => ["T", "T/3", "D7", "D7-1", "D7/3", "D7/5", "D7/7", "Ts"],
        "D7-5" => ["T", "D7-5"],
        "S65"  => ["T/5", "D", "D64", "S65"],
        "S65/6"=> ["D", "D7", "D7-1", "D7-5", "T/5", "S65/6"],
        "S6"   => ["T/3", "D", "D7", "D7-5", "D64", "S6"],
        "Ts"   => ["T/3", "S", "S/3", "S6", "S65", "S65/6", "D", "D-5", "D7", "D7-1", "Ts", "Tm"],
        "Tm"   => ["S", "D/3", "D7/3", "Ts", "Tm"],
        "Dm"   => ["S/3", "Ts", "Dm"])
        # ["T", "T-5", "T/3", "T/5", "S", "S/3", "S/5", "S6", "S65", "S65/6", "D", "D-5", "D/3", "D/5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "T64", "T54", "D64", "D54", "Ts", "Tm", "Dm"]
    end
        
    return valid_function_table
end

createValidFunctionTable (generic function with 1 method)

# # Rhythm function

In [10]:
function rhythmify(time_signature, original) # (beats per bar, subdivisions) (3,1), (3, 2), (4,1), (4, 2)
    
    rhythm_map = String[]
    
    beat_remainder = size(original, 2) % (time_signature[1] * time_signature[2])
    
    rhythms = Dict(
        (3, 1) => ["S", "W", "W"],
        (3, 2) => ["S", "O", "W", "O", "W", "O"],
        (4, 1) => ["S", "W", "S", "W"],
        (4, 2) => ["S", "O", "W", "O", "S", "O",  "W", "O"]
    )

    ## Create basic rhythm map NOT USED
    #=
    if beat_remainder == time_signature[2]
        for i in 1:time_signature[2]
            push!(rhythm_map, rhythms[time_signature][end - (time_signature[2] - i)])
        end
            
        for i in 0:(size(original, 2) - 1-time_signature[2])
            push!(rhythm_map, rhythms[time_signature][(i % (time_signature[1]*time_signature[2]) + 1)])
        end
    else
        for i in 0:(size(original, 2) - 1)
            
            push!(rhythm_map, rhythms[time_signature][(i % (time_signature[1]*time_signature[2]) + 1)])
        end
    end
    =#
    
    if time_signature == (4,2)
        if size(original, 2) == 16
            rhythm_map = ["S", "O", "W", "O", "S", "C",  "C", "C", "S", "O", "W", "O", "S", "O",  "W", "O"]
            rhythm_map = ["S", "W", "S", "W", "S", "C",  "C", "C", "S", "W", "S", "W", "S", "W",  "S", "W"]
        elseif size(original, 2) == 17
            rhythm_map = ["O", "S", "O", "W", "O", "S", "O",  "W", "O", "S", "O", "W", "O", "S", "O",  "W", "O"]
            rhythm_map = ["W", "S", "W", "S", "W", "S", "W",  "S", "W", "S", "W", "S", "W", "S", "W",  "S", "W"]
        end
    #=
    elseif time_signature == (3,2)
        if size(original, 2) == 12
            rhythm_map = ["S", "O", "W", "O", "W", "O", "S", "O", "W", "O", "W", "O"]
        elseif size(original, 2) == 13
            rhythm_map = ["O", "S", "O", "W", "O", "W", "O", "S", "O", "W", "O", "W", "O"]
        end
    =#
    else
        println("Time Signature: only (4,2) is supported at this point")       
    end
    return rhythm_map
end

rhythmify (generic function with 1 method)

In [11]:
# Child.sequence : [1:(1:function1_cost, 2:"function1_name", 3:[function1_voicing_list]), 2:(1:function2_cost, 2:"function2_name", , 3:[function2_voicing_list]), ...]
function function_cost(candidateSequence, rhythm_map, valid_function_table, debug=false)

    child = candidateSequence.sequence
    total_cost = candidateSequence.function_cost

    current_child = child[end]
    current_function = current_child[2]
    current_index = size(child, 1)
    previous_child = child[end-1]
    previous_function = previous_child[2]
    previous_index = size(child, 1) - 1
    timestep = size(child, 1)
    
    if size(candidateSequence.last_four, 1) < 4
        push!(candidateSequence.last_four, child[end][2])
    else
        popfirst!(candidateSequence.last_four)
        push!(candidateSequence.last_four, child[end][2])
            
        if size(unique(candidateSequence.last_four), 1) < 3
            total_cost += 1000
        end
    end
    total_cost += current_child[1]

        
    if !(current_function in valid_function_table[previous_function])
        total_cost += 1000
    end
     
    # General cadences
        
    if rhythm_map[timestep-1] == "C" && rhythm_map[timestep] != "C"
       if debug
            println("Remember the cadence from last time and not repeat it")
        end
        candidateSequence.last_cadence = previous_function
    end
        
    if rhythm_map[timestep] == "C" && timestep == size(rhythm_map, 1)
           
        if candidateSequence.last_cadence == current_function
            if debug 
                println("Avoid same cadence")
            end
            total_cost += 1000
        end
    end
    
    if rhythm_map[timestep] == "C" && (timestep == size(rhythm_map, 1) || timestep == size(rhythm_map, 1)-1)
        if !(current_function == "T" || current_function == "D" || current_function == "Ts")
            total_cost += 1000
            if debug
                println("Intermediate cadences must end with T, D or Ts")
            end
        end
        #=
        if previous_function != current_function && timestep == size(rhythm_map, 1)
              total_cost += 1000
            if debug
                println("Intermediate cadence at end of segment must hold its final function")
            end
        end
        =#
    end
            
    if rhythm_map[timestep] == "C" && rhythm_map[timestep-1] != "C"
        if !(string(current_function[1]) == "D" || current_function == "T" || current_function == "S" || current_function == "S/3" || current_function == "Ts")
            total_cost += 1000
            if debug
                println("Intermediate cadences must begin with T, S, S/3 or (any) D")
            end
        end      
    end
    
    if rhythm_map[timestep] == "C" && rhythm_map[timestep-1] == "C"
        if string(previous_function[1]) == "D"
            if !(current_function == "T" || current_function == "Ts" || string(current_function[1]) == "D")
                total_cost += 1000
                if debug
                    println("C: D? must go to T, Ts or D")
                end
            end
            if string(child[end-2][2][1]) == "D" && string(child[end][2][1]) == "D"
                total_cost += 1000
                if debug
                    println("C: Cadence can't be all D's")
                end
            end  
        end

        if previous_function == "T" 
            if !(current_function == "T" || current_function == "S" || current_function == "D")
                total_cost += 1000
                if debug
                    println("C: T-> T, S or D")
                end
            end
            if child[end-2][2] == "T" && child[end][2] == "T"
                total_cost += 1000
                if debug
                    println("C: Cadence can't be all T's")
                end
            end  
        end

        if previous_function == "S" 
            if !(current_function == "D" || current_function == "S")
                total_cost += 1000
                if debug
                    println("C: S-> S or D")
                end
            end
            if child[end][2] == "S" && child[end-2][2] == "S"
                total_cost += 1000
                if debug
                    println("C: Cadence can't be all S's")
                end
            end  
        end

        if previous_function == "S/3" 
            if !(current_function == "D" || current_function == "S/3")
                total_cost += 1000
                if debug
                    println("C: S/3-> S/3 or D")
                end
            end
        end
        if child[end][2] == "S/3" && child[end-2][2] == "S/3"
            total_cost += 1000
            if debug
                println("C: Cadence can't be all S/3's")
            end

        end

        if previous_function == "Ts" 
            if !(current_function == "D" || current_function == "Ts")
                total_cost += 1000
                if debug
                    println("C: Ts-> Ts or D")
                end
            end
        end
        if child[end][2] == "Ts" && child[end-2][2] == "Ts"
            total_cost += 1000
            if debug
                println("C: Cadence can't be all Ts's")
            end

        end
        #=
        if rhythm_map[timestep-2] == "C" && previous_function == child.sequcence[end-2][2]
            total_cost += 1000
            if debug
                println("C: Cadence can't rest on first chord")
            end
        end
        =#
    end
        
   
    if rhythm_map[timestep] != "C" && rhythm_map[timestep-1] == "C"
        if previous_function != child[end-2][2] 
            total_cost += 1000
            if debug
                println("C: Cadence must rest on final function mid phrase")
            end
        end
    end
        
    if rhythm_map[timestep] == "C" && timestep == size(rhythm_map, 1) && previous_function != current_function
        total_cost += 1000
        if debug
                println("C: Cadence must rest on final function at the end")
        end
    end
    

    # Authentic Cadences
    if rhythm_map[timestep] == "A"
    
        if (timestep == size(rhythm_map, 1) || timestep == size(rhythm_map, 1)-1) && !(current_function == "T")
            total_cost += 1000
            if debug
                println("A: Cadence must rest on final function T")
            end            
        end
            
        if !((current_function in Dominants) || current_function == "T" || current_function == "S")
            total_cost += 1000
            if debug
                println("A: Cadence can only contain T, S, or any D")
            end    
        end 
            
        if (timestep == size(rhythm_map, 1)-2) && !(current_function in Authentic_Dominants)
            total_cost += 1000
            if debug
                println("A: Cadence must end on D->T")
            end    
        end 
        
        if rhythm_map[timestep-1] != "A" && string(current_function)[1] == string(previous_function)[1]
            total_cost += 1000
            if debug
                println("A: Avoid leading into the cadence with a chord repetition")
            end   
        end
        
        if rhythm_map[timestep-1] != "A" && current_function == "T"
           total_cost += 1000
            if debug
                println("A: Avoid starting cadence with T")
            end
        end
    end
    
    # Chord repetition can only happen from stronger to weaker beats. Otherwise, cost is set by distance to original tonal_map.
    if (previous_function == current_function) #&& !((rhythm_map[timestep] == "A" || rhythm_map[timestep-1] == "A") || (rhythm_map[timestep] == "C" || rhythm_map[timestep-1] == "C"))
        if (beat_accentuation[rhythm_map[timestep]] > beat_accentuation[rhythm_map[timestep-1]])
            if debug
                println("Chord repetition can only happen from stronger to weaker beats. Otherwise, cost is set by distance to original tonal_map.")
            end
            total_cost += 1000
        end
    end
    
    # /5 chords     
    # T5, S5, D5 requires same function type before and after
    if (previous_function == "T/5" || previous_function == "S/5" || previous_function == "D/5")
        if string(child[end-2][2][1]) != string(current_function[1])
            if debug
                println("T/5, S/5, D/5 requires same function type before and after")
            end

            total_cost += 1000
        end   
    end

        
    ## Suspended chords
        
    # Augmented chords must lie on a more accentuated beat than its resolution
    
    if (current_function == "T64" || current_function == "D64") && !(rhythm_map[current_index] == "S" || previous_function == current_function || rhythm_map[current_index] == "A")
        if debug
            println("T64, D64 must begin on strong beat")
        end
        total_cost += 1000
    end
    
    
    # Ts -> S/3 or Ss -> D7-1 must decrease in rhythmic accentuation
    if ((previous_function == "Ts" && current_function == "S/3") || (previous_function == "Ss" && current_function == "D7-1")) && 
        beat_accentuation[rhythm_map[previous_index]] < beat_accentuation[rhythm_map[current_index]]
        if debug
            println("Ts -> S/3 or Ss -> D7-1 must decrease in rhythmic accentuation")
        end
        total_cost += 1000
    end
    
    return total_cost
end

function_cost (generic function with 2 methods)

In [12]:
## Function for calculating the voicing cost between neighbouring voicings
# Child.sequence : [1:(1:function1_cost, 2:"function1_name", 3:[function1_voicing_list]), 2:(1:function2_cost, 2:"function2_name", , 3:[function2_voicing_list]), ...]
function voicing_cost(candidateSequence, rhythm_map_local, data, full=false, debug=false)
    
    child = candidateSequence.sequence
    
    if full
        total_cost = 0
        from = 2
    else
        from = size(child, 1)
        total_cost = candidateSequence.voicing_cost
    end
        
    # OPTIONAL: distance from original cost
    #total_cost += distance_cost(data, child, size(child, 1), false)
        
    if size(candidateSequence.last_soprano, 1) < 4
        push!(candidateSequence.last_soprano, child[end][3][4][1])
    else
        popfirst!(candidateSequence.last_soprano)
        push!(candidateSequence.last_soprano, child[end][3][4][1])        
    end
            
    if size(candidateSequence.last_soprano, 1) == 4    
        if size(unique(candidateSequence.last_soprano), 1) < 3
            total_cost += 1000
        end
    end
        
    for i in from:size(child, 1)
        
        current_notes = child[i][3]
        current_function = child[i][2]
        current_index = size(child, 1)
        previous_notes = child[i-1][3]
        previous_function = child[i-1][2]
            
        dist = [current_notes[1][1] - previous_notes[1][1], current_notes[2][1] - previous_notes[2][1], current_notes[3][1] - previous_notes[3][1], current_notes[4][1] - previous_notes[4][1]]

        for (j, d) in enumerate(dist)
            if d == 0
                candidateSequence.same_note[j] += 1
            else
                candidateSequence.same_note[j] = 0
            end
        end

        if maximum(candidateSequence.same_note) > 3
            total_cost += 1000
            if debug
                println("Same note held for too long in one of the voices")
            end
        end
        if i > 2
            ancient_notes = child[i-2][3]
            ancient_function = child[i-2][2]
            previous_dist = [previous_notes[1][1] - ancient_notes[1][1], previous_notes[2][1] - ancient_notes[2][1], previous_notes[3][1] - ancient_notes[3][1], previous_notes[4][1] - ancient_notes[4][1]]
               
            # bass stuff
            if abs(previous_dist[1]) > 2 && !(sign(previous_dist[1]) != sign(dist[1])) && abs(dist[1]) > 2
                if debug
                    println("Leaps in bass should be countered by a move in the other direction")
                end
                total_cost += 2              
            elseif !(sign(previous_dist[1]) == sign(dist[1])) || sign(dist[1]) == sign(0) #|| !(previous_function in Dominants_dependent || previous_function in Tonics_dependent)
                if debug
                    println("Encourage walking bass")
                end
                total_cost += 2              
            end
                        
            if abs(dist[1]) == 0 && !(previous_function in Dominants_dependent || previous_function in Tonics_dependent || rhythm_map_local[i] == "C" || rhythm_map_local[i] == "A") 
                if debug
                    println("Avoid dead bass")
                end
                total_cost += 2              
            end 
                    
            if abs(previous_dist[1]) > 2 && abs(dist[1]) > 2
                if sign(previous_dist[1]) == sign(dist[1])
                    if debug
                        println("Leaps in bass should be countered by a step-wise move in the other direction")
                    end
                    total_cost += 2
                end
                total_cost += 2              
            end
                          
            if abs(previous_dist[4]) > 2 && abs(dist[4]) > 2 
                if sign(previous_dist[4]) == sign(dist[4])
                    if debug
                        println("Leaps in soprano should be countered by a step-wise move in the other direction")
                    end
                    total_cost += 4
                end
                total_cost += 2             
            end
            
            # MAYBE soprano should be encouraged to move in the same direction
            #=
            if sign(dist[4]) != sign(previous_dist[4]) && sign(dist[4]) != 0
                total_cost += 4        
            end
            =#
            #=            
            # Encourage soprano to not switch back and forth
            if abs(dist[1]) > 2
                total_cost += 2         
            end
            =#            
            #=
            if abs(dist[4]) > 2 || dist[4] == 0
                total_cost += 2         
            end 
            =#
        end

        ### Voice specific rules
        
        # Avalanche check
        if abs(reduce(+, map(x -> sign(x), dist))) == 4
            if (rhythm_map_local[i] == "A" ||  rhythm_map_local[i] == "C") && reduce(+, map(x -> sign(x), dist)) == -4 && (abs(dist[4]) < 3)
                total_cost = total_cost
            else
                if debug
                    println("Avalanche")
                end
                total_cost += 1000
            end
        end
          
        # Immediately return if cost > 1000
        if total_cost > 999 && !debug
            #candidateSequence.voicing_cost += total_cost
            return total_cost 
        end

        # voice movement cost = sum of all voices movement IF it exceeds 2 (step-wise motion has no cost)
        total_cost += reduce(+, map(x -> abs(x) > 2 ? 3 : 0 , dist[2:3])) # Flat penalty for leaps in T, A
        total_cost += reduce(+, map(x -> abs(x) > 2 ? 2 : 0 , dist[4])) # Flat penalty for leaps in S
        total_cost += reduce(+, map(x -> abs(x) > 2 ? 4 : 0 , dist[1])) # Flat penalty for leaps in B,
              
        # Discourage voicing repetition ? 
        if (reduce(+, map(x -> abs(x), dist)) == 0)
                    
            if !(rhythm_map_local[i] == "A") #|| rhythm_map_local[i] == "C")
                if debug
                    println("Repetition")
                end
                total_cost += 1000
            end
                
            if rhythm_map_local[i] == "A" && !(i == size(rhythm_map_local, 1))
                if debug
                    println("Avoid repetition in authentic cadence until end")
                end
                total_cost += 4             
            end
        end
                
        # Discourage doubling of the same MIDI note
        active_notes = []
        for voice in current_notes
            push!(active_notes, voice[1])         
        end
        #println(active_notes)
        if size(unique(active_notes), 1) != 4
            total_cost += 5
            if debug
                println("Voices doubling the same MIDI note")
            end
        end
        
        # Encourage opposite movement between bass and soprano
        if sign(dist[1]) == sign(dist[4]) #&& dist[1] != 0
            total_cost += 2
        end
             
        # Mid voices must move < 8
        if !(dist[2] < 8) || !(dist[3] < 8)
            if debug
                println("Mid voices must move < 8")
            end
            total_cost += 1000
        end
        

        # Outer voices should only move < 8
        if !(abs(dist[1]) < 8) || !(abs(dist[4]) < 8)
            total_cost += 2
        end

            
        # Outer voices may move < 9 or 12 at a cost
        if (abs(dist[1]) >= 8 && abs(dist[1]) != 12) || (abs(dist[1]) > 7 && abs(dist[1]) < 10 && previous_function != current_function) || (abs(dist[4]) >= 10 && abs(dist[4]) != 12)
            if debug
                println("Outer voices may move < 9 or 12 at a cost")
            end
            total_cost += 1000
        end

        # Parallell primes, fifths and octaves are illegal (bevege seg samme retning i 0, 7)
        if current_notes != previous_notes
            pairs = [(1, 2), (2, 3), (3, 4), (1, 3), (2, 4), (1, 4)]
            for (voice1, voice2) in pairs
                if sign(dist[voice1]) == sign(dist[voice2]) && dist[voice1] != 0
                    if abs(current_notes[voice1][1] - current_notes[voice2][1]) % 12 == 7 && abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 7
                        total_cost += 1000
                        if debug
                            println("Parallell fifth")
                        end
                    elseif abs(current_notes[voice1][1] - current_notes[voice2][1]) % 12 == 0 && abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 0
                        total_cost += 1000
                        if debug
                            println("Parallell octave")
                        end
                    # Parallell movement from diminished fifth to fifth is illegal
                    elseif abs(current_notes[voice1][1] - current_notes[voice2][1]) % 12 == 7 && 
                            abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 6
                        total_cost += 1000
                        if debug
                            println("Parallell tritone to fifth")
                        end                          
                    end
                end
            end
        end
        
        # Hidden parallell fifths and octaves are illegal WORKS!
        if (sign(dist[1]) == sign(dist[4])) && abs(dist[4]) > 2 && ((abs(current_notes[1][1] - current_notes[4][1]) % 12 == 0) || (abs(current_notes[1][1] - current_notes[4][1]) % 12 == 7))
            if debug
                println("Hidden parallell fifths and octaves are illegal")
            end
            total_cost += 1000
        end
                                
        # Immediately return if cost > 1000

        if total_cost > 999 && !debug
            #candidateSequence.voicing_cost += total_cost
            return total_cost
        end
        
        ### Chord specific rules
        ## Dominants               
        
        # From S to D, bass moves stepwise up while rest move shortest path down
        if (previous_function == "S" && current_function == "D") && dist[1] != 2 && !(rhythm_map_local[i] == "A" || rhythm_map_local[i] == "C")
            if debug
                    println("From S to D, bass moves stepwise up while rest move shortest path down")
                end
            total_cost += 1000
        end
                        
        #if total_cost > 999 && !debug
            #candidateSequence.voicing_cost += total_cost
           #return total_cost 
        #end
                        
        # Leading note in D goes to root, except can leap down a 3rd to 5 in mid voices # WORKS ?           
        if string(previous_function[1]) == "D" && (string(current_function[1]) == "T" && !(current_function == "Ts") && !(current_function == "Tm"))
               
            for (j, voice) in enumerate(previous_notes)
                                
                if rhythm_map_local[i] == "A" || rhythm_map_local[i] == "C" 
                    if voice[2] == 3 && !((current_notes[j][2] == 1 && dist[j] == 1) || 
                    (((current_notes[j][2] == 5 && dist[j] == -4)) && (j == 2 || j == 3) && (rhythm_map_local[current_index] == "A" || rhythm_map_local[current_index] == "C")))
                        if debug
                            println("The leading note must move to 1 (or down to 5 in cadence) in T")
                        end

                        total_cost += 1000
                    end
                elseif voice[2] == 3 && current_notes[j][2] != 1 && dist[j] != 1
                    if debug
                        println("D->T: D:3 must resolve into T:1")
                    end
                    total_cost += 1000
                end
            end
                            
        end
                        
        ## D7
                                
        if (previous_function == "D7" || previous_function == "D7/3" ||  previous_function == "D7/5" || 
                                    previous_function == "D7/7" || previous_function == "D7-5") && !(current_function == "Ts")              
            for (j, voice) in enumerate(previous_notes)
                
                if voice[2] == 3 && !((current_notes[j][2] == 1 && (dist[j] == 1 || dist[j] == 2)) || 
                    (((current_notes[j][2] == 5 && dist[j] == -4)) && (j == 2 || j == 3) && (rhythm_map_local[current_index] == "A" || rhythm_map_local[current_index] == "C")))
                    if debug
                        println("D7: The leading note must resolve into 1 (or 5 if cadence) in T") # tested
                    end
                    total_cost += 1000
                end
                if voice[2] == 7 && !(current_notes[j][2] == 3 && (dist[j] == -2 || dist[j] == -1))
                    if debug
                        println("D7: The 7 must resolve into 3 in T") # tested
                    end
                    total_cost += 1000
                end
                if voice[2] == 1 && !((current_notes[j][2] == 1) || (current_notes[j][2] == 5))
                    if debug
                        println("D7: The 1 must go to 1 or 5 in T") # tested
                    end
                    total_cost += 1000
                end
                
            end                
        end
                                  
        # D7/5: bassen må føres trinnvis inn og ut av akkorden, kvarten må føres med tonegjentakelse inn og ut
        if previous_function == "D7/5"
            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 1 && !(dist[j] == 0)
                    if debug
                        println("D7/5: The root must remain stationary") # tested
                        
                    end
                    total_cost += 1000
                end                       
            end  
                                            
            if abs(dist[1]) > 2 
                                            
                if debug
                    println("D7/5: The bass must move step-wise throughout the chord") # tested
                end
                total_cost += 1000
            end                
        end
                                        
        if current_function == "D7/5"

            for (j, voice) in enumerate(current_notes)
                if voice[2] == 1 && !(dist[j] == 0)
                    if debug
                        println("D7/5: The root must remain stationary") # Tested
                        
                    end
                    total_cost += 1000
                end                       
            end  
                                            
            if abs(dist[1]) > 2 
                                            
                if debug
                    println("D7/5: The bass must move step-wise throughout the chord") # Tested
                end
                total_cost += 1000
            end                                                             
        end
                                
        #D7-1
        
        if previous_function == "D7-1" && !(current_function == "Ts" || current_function == "D7" || current_function == "D7/3" || current_function == "D7/5" || 
                                            current_function == "D7/7" || current_function == "D7-1")
            sevencounter = 0
            sevenvoice = []
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 3 && !((current_notes[j][2] == 1 && (dist[j] == 1 || dist[j] == 2)) || 
                    (((current_notes[j][2] == 5 && dist[j] == -4)) && (j == 2 || j == 3) && (rhythm_map_local[current_index] == "A" ||rhythm_map_local[current_index] == "C" )))
                    if debug
                        println("D7-1: The leading note must resolve into 1 (or 5 if cadence) in T") # tested
                    end
                    total_cost += 1000
                end
                # The 7 in D7 must either go step-wise down or up
                if voice[2] == 7
                    sevencounter += 1
                    push!(sevenvoice, j)
                    if !(current_notes[j][2] == 3 && (dist[j] == -2 || dist[j] == -1) || 
                                    current_notes[j][2] == 5 && dist[j] == 2)
                        if debug
                            println("D7-1: The 7 must resolve step-wise into 3 or 5 in T") # tested
                        end
                        total_cost += 1000
                    end
                end                
            end

            if sevencounter == 2 && (current_notes[sevenvoice[1]][2] == current_notes[sevenvoice[2]][2])
                if debug
                    println("D7-1: doubled septims must move in opposite directions")
                end
                total_cost += 1000
            end
        end
                            
        ## /5 chords WORKS
                        
        # Bass and voice with /5 root must move max step-wise in and out.
        if current_function == "T/5" || current_function == "D/5" || current_function == "S/5" 

            if abs(dist[1]) > 2 
                total_cost += 1000  
            end
                            
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 1 && abs(dist[j]) > 2
                    total_cost += 1000
                end
            end
        end
                        
        if previous_function == "T/5" || previous_function == "D/5" || previous_function == "S/5"
            if abs(dist[1]) > 2 
                total_cost += 1000 
            end
                            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 1 && abs(dist[j]) > 2
                    total_cost += 1000
                end
            end
        end
        
        ## Augmented chords
                        
        if current_function != previous_function
            # The 4 in 64 chords must be introduced stationary and resolved step-wise                
            if current_function == "T64" || current_function == "D64"

                for (j, voice) in enumerate(current_notes)
                    if voice[2] == 4 && dist[j] != 0
                        if debug
                            println("4 is not stationary into augmented")
                        end
                        total_cost += 1000
                    end
                    if voice[2] == 6 && abs(dist[j]) > 2
                        if debug
                            println("6 leaps into augmented")
                        end
                        total_cost += 1
                    end
                end
            end
        end
        if (previous_function == "T64" || previous_function == "D64") && current_function != previous_function 
            if dist[1] != 0
                total_cost += 1000
            end
            # If same function class two timesteps ago
            if string(current_function[1]) == string(ancient_function[1])
                total_cost += 1000
            end


            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 4 && !(abs(dist[j]) < 3) && !(current_function == "T54" || current_function == "D54")
                    if debug
                        println("4 is not resolved step-wise down")
                    end
                    total_cost += 1000
                end
                if voice[2] == 6 && !(abs(dist[j]) < 3)
                    if debug
                        println("6 is not resolved step-wise down")
                    end
                    total_cost += 1000
                end
                if (voice[2] == 1 && dist[j] != 0) 
                    if debug
                        println("Root notes must be stationary")
                    end
                    total_cost += 1000                      
                end
            end
        end
                                    
        # The 4 in T54 must be introduced stationary and resolved step-wise down
        if (current_function == "T54" || current_function == "D54") 

            for (j, voice) in enumerate(current_notes)
                if voice[2] == 4 && dist[j] != 0
                    if debug
                        println("4 is not stationary in augmented")
                    end
                    total_cost += 1000
                end
            end
        end

        if (previous_function == "T54" || previous_function == "D54") && current_function != previous_function
            if dist[1] != 0
                if debug
                    println("T54/D54: Bass must be stationary")
                end
                total_cost += 1000 
            end

            #= 
            if string(current_function[1]) == string(ancient_function[1]) && !(ancient_function == "D64" || ancient_function == "T64")
                if debug
                    println("T54/D54: Not same function before and after")
                end
                total_cost += 1000
            end
            =#

            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 4 && !(abs(dist[j]) < 3)
                    if debug
                        println("4 is not resolved step-wise down")
                    end
                    total_cost += 1000
                end
                if (voice[2] == 1 && dist[j] != 0) || (voice[2] == 5 && dist[j] != 0)
                    if debug
                        println("Root and fifth notes must be stationary")
                    end
                    total_cost += 1000                      
                end
            end

        end

                                
                            
        ## /3 chords
                                
        if (previous_function == "T/3" && current_function == "D/3") || (previous_function == "S/3" && current_function == "T/3")                    
            if sign(dist[1]) != sign(-1)                              
                if debug
                    println("T/3 -> D/3 or S/3 -> T/3 needs to have bass leaping downwards")
                end
                total_cost += 1000
            end
        end
                                
        if (previous_function == "T/3" && current_function == "S/3")                  
            if sign(dist[1]) != sign(1)                              
                if debug
                    println("T/3 -> S/3 needs to have bass leaping upwards")
                end
                total_cost += 1000
            end
        end
       
        # /3 chords together are encouraged, but only if correct voicing is used
        if i > 2 && (previous_function == "T/3" || previous_function == "S/3" || previous_function == "D/3")

            if (ancient_function == "T/3" && previous_function == "D/3") || 
                (ancient_function == "S/3" && previous_function == "T/3") || 
                (ancient_function == "T/3" && previous_function == "S/3")
                #total_cost -= 1
                                
                # Leaps in bass should be countered by a move in the other direction
                if (abs(dist[1]) > 2)
                    if debug
                        println("Two /3 chords with a leap in bass need to be countered by a stepwise motion")
                    end
                    total_cost += 1000          
                end
                if  (sign(previous_dist[1]) == sign(dist[1]))
                    if debug
                        println("Two /3 chords with a leap must be resolved with motion in the other direction")
                    end
                    total_cost += 1000
                end
                if (sign(previous_dist[1]) == sign(previous_dist[4]))
                    if debug
                        println("Only use two /3 chords in motion when soprano is moving opposite to the bass or is flat")
                    end
                    total_cost += 1000
                end

            end
        end
                                    
                                     
        ## Mediants
                                    
        # The 3 of D becomes 3 of Ts
                               
        if ((previous_function == "D" || previous_function == "D7"  || previous_function == "D7-1")  && current_function == "Ts")  
                                        
           for (j, voice) in enumerate(previous_notes)
                if voice[2] == 3 && current_notes[j][2] != 3
                    if debug
                        println("D->Ts: 3 in D must resolve into 3 in Ts")
                    end
                    total_cost += 1000
                end
            end                                     
        end
                               
        # Increment candidateSequence.num_mediants if current_chord is mediant
        if (current_function == "Ts" || current_function == "Ss" || current_function == "Dm" || current_function == "Tm")
            candidateSequence.num_mediants += 1
        end
        
        # No more than 4 mediants per 16 timesteps
        if candidateSequence.num_mediants > 2
            if debug
                println("Too many mediants in the sequence.")
            end
            total_cost += 1000
        end
                                    
        # S65
                                    
        if current_function == "S65" || current_function == "S65/6"
                                        
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 6 && (abs(dist[j]) > 2)
                    if debug
                        println("S65: 6 must be introduced step-wise")
                    end
                    total_cost += 1000
                end
                if voice[2] == 5 && (abs(dist[j]) != 0)
                    if debug
                        println("S65: 5 must be introduced stationary")
                    end
                    total_cost += 1000
                end

            end     

        end
                                    
        if previous_function == "S65"
            fifth = 0
            sixth = 0
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 6
                    sixth = j

                end
                if voice[2] == 5
                    fifth = j
                end
            end     

            if !(abs(dist[fifth]) == 0 && abs(dist[sixth]) < 3 || abs(dist[sixth]) == 0 && abs(dist[fifth]) < 3) 

                total_cost += 1000
                if debug
                    println("S65: 5 and 6 must be resolved in oblique motion")
                end
            end
        end
                                    
        if previous_function == "S65" && rhythm_map_local[size(child, 1) - 1] == "W"
                                        
            if !(current_function == "D64" || current_function == "T/5")
                total_cost += 1000
                if debug
                    println("S65: On weak beat must resolve into D64 or T/5")
                end 
            end
        end
                                                                
        if current_function == "S6"
                                       
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 6 && (abs(dist[j]) > 2)
                    if debug
                        println("S6: 6 should be introduced step-wise")
                    end
                    total_cost += 2
                end   

            end
        end
                                    
       if previous_function == "S6" 
                                       
            for (j, voice) in enumerate(current_notes[2:4])
                if sign(dist[j]) != sign(-1) || dist[j] != 0
                    if debug
                        println("S6: Upper voices must resolve downwards")
                    end
                    total_cost += 1000
                end   

            end
        end                             
                
        ## Enforce spread chord on last (counting) timestep            
        if rhythm_map_local[size(child, 1)] == "A" && (size(child, 1) == size(rhythm_map_local, 1)-1)
                                      
            current_notes = deepcopy(child[i][3])
            list = []
            for i in 1:4
                push!(list, current_notes[i][1])
            end
            if size(unique(list), 1) != 4
                total_cost += 1000
                if debug
                    println("Final chord should be spread")
                end
            end
                                                                
            #=
            if sign(dist[1]) != sign(-1)
               total_cost += 1000
                if debug
                    println("Final chord should have descending bass")
                end                         
            end
            =#
        end
                                    
        if rhythm_map_local[size(child, 1) - 1] == "C" && rhythm_map_local[size(child, 1)] != "C" && dist[4] == 0
            total_cost += 1000
                if debug
                    println("C: Soprano should not rest after a cadence.")
                end
        end      
                                    
        if rhythm_map_local[size(child, 1)-1] == "C" && rhythm_map_local[size(child, 1)] == "C" && dist[4] != 0
            total_cost += 4
                if debug
                    println("C: Soprano rest in a cadence.")
                end
        end                                
                                       
    end                                 
                                        
    return total_cost
        
end

voicing_cost (generic function with 3 methods)

In [13]:
# Rate the voicing permutation with regards to the original score
function distance_cost(data, voicing_permutation, original_timestep, debug=false)

    local_dist = 0
    voices = voicing_permutation[end][3]
    active_notes = getActiveNotes(data, original_timestep)
    
    if debug
        println("Active notes: ", active_notes)
        println("Voicing: ", voices)
    end
    
    if isempty(active_notes)
        return local_dist
    end
    for (i, voice) in enumerate(voices)
        
        if i == 1
            local_dist += abs(active_notes[1] - voice[1])
            if debug
                println("Bass distance: ", abs(active_notes[1] - voice[1]))
            end
        #=
        elseif i == 2 || i == 3
            if debug
                println("Mid-voice distance: ", minimum([abs(active_note - voice[1]) for active_note in active_notes]))
            end
            total_dist += minimum([abs(active_note - voice[1]) for active_note in active_notes])
        =#
        elseif i == 4
            local_dist += abs(active_notes[end] - voice[1])
            if debug
                println("Soprano distance: ", abs(active_notes[end] - voice[1]))
            end
        end
    end
    
    # To normalize voicing costs, divide by 12 and round to nearest integer
        
    local_dist = floor(Int, local_dist / 12)
            
    return local_dist
 
end

distance_cost (generic function with 2 methods)

In [14]:
# Build MinList as we go, for every function sequence pruned at depth i or completed, update its cost-to-end in 2-D table per timestep
function initialiseBetterHeuristic(function_permutations, num_timesteps)
    
    num_functions = size(function_permutations, 1)
    
    estimatorcontainer = fill(Int16(-1), (num_timesteps-1, num_functions, num_functions, num_functions))

    voicingindeces = Dict{Array{Tuple{UInt16,UInt16}, 1}, Integer}()
    for (i, voicing) in enumerate(function_permutations)
        voicingindeces[voicing] = i
    end
   
    return estimatorcontainer, voicingindeces
end

estimatorcontainer = []
    


0-element Array{Any,1}

# # DFS Branch and Bound ALL IN ONE

In [15]:
function prepare(data, time_signature, functions, function_potential, function_potential_unsorted, 
        voicing_permutations, voicing_permutations_raw, valid_function_table, phase, previous_sequence)
        
    leading_end = Dominants
    start = ["T"]

    function_potential = function_potential[1:size(data, 2)]
    voicing_permutations = voicing_permutations
    rhythm_map = rhythmify(time_signature, data)
    
    # Locate and enforce important events (start, cadences)
    if phase == 1
        start_index = findfirst(x -> x == "S", rhythm_map) 
        #final_index = findlast(x -> x == "S" || x == "W", rhythm_map)
        final_index = findlast(x -> x == "S", rhythm_map)
        #cadence_index = findlast(x -> x == "W" || x == "S", rhythm_map[1:final_index-1])  
        cadence_index = findlast(x -> x == "W", rhythm_map[1:final_index-1])  
         
        # Set start as the Tonic
        filter!(x -> x[2] in start, function_potential[start_index])
            
        for i in cadence_index:size(data, 2)
            rhythm_map[i] = "C" # Any cadence
        end

    elseif phase == 2
        # Set last two functions as tonic for ending
        filter!(x -> x[2] in start, function_potential[end])
        filter!(x -> x[2] in start, function_potential[end-1])
        filter!(x -> x[2] in ["D", "D7"], function_potential[end-2])
            
        final_index = findlast(x -> x == "S", rhythm_map)
        #cadence_index = findlast(x -> x == "S" || x == "W", rhythm_map[1:final_index-1]) 
        cadence_index = findlast(x -> x == "S", rhythm_map[1:final_index-1]) 
        #cadence_index = final_index
        for i in cadence_index:size(data, 2)
            rhythm_map[i] = "A" # Authentic cadence
        end
    end
               
    println("Rhythm map: ", rhythm_map, " size: ", size(rhythm_map))
    
    
    # Prune function_potential by beat restrictions for each chord
    for (i, potential) in enumerate(function_potential)
        for (j, (cost, chord)) in enumerate(potential)
            if !(rhythm_map[i] in functions[chord][4])
                filter!(x -> x[2] != chord, potential)
            end
        end
    end
        
    activeset = candidateSequence[]
    score = []
    if phase == 1
        # Func : (cost, function) Find and sort the voicings for the first timestep
        for (cost, func) in function_potential[1]
            for potential_voicing in voicing_permutations[func]
                temp = [(cost, func, potential_voicing)]
                local_cost = distance_cost(data, temp, 1, false)
                temp = [local_cost, (cost, func, potential_voicing)]
                push!(score, temp)
            end
        end
        
         # Find best starting points
        score = sort!(score, by = x->x[1], rev=true)
        best_score = score[end][1]
        for entry in score
            if entry[1] == best_score
                start_point = [entry[2]]
                startCandidate = candidateSequence()
                startCandidate.sequence = start_point
                startCandidate.function_cost = 0
                startCandidate.voicing_cost = 0
                startCandidate.num_mediants = 0
                startCandidate.total_cost = 0
                startCandidate.total_cost_per_timestep = [0 for x in 1:size(data, 2)]
                startCandidate.function_cost_per_timestep = [0 for x in 1:size(data, 2)]
                startCandidate.same_note = [0, 0, 0, 0]
                startCandidate.last_four = ["T"]
                startCandidate.last_soprano = []
                startCandidate.last_cadence = "X"
                push!(startCandidate.last_soprano, startCandidate.sequence[end][3][4][1])
                push!(activeset, startCandidate)
            end
        end
    else
        startCandidate = candidateSequence()
        startCandidate.sequence = previous_sequence
        startCandidate.function_cost = 0
        startCandidate.voicing_cost = 0
        startCandidate.num_mediants = 0
        startCandidate.total_cost = 0
        startCandidate.total_cost_per_timestep = [0 for x in 1:size(data, 2)]
        startCandidate.function_cost_per_timestep = [0 for x in 1:size(data, 2)]
        startCandidate.same_note = [0, 0, 0, 0]
        startCandidate.last_four = []
        startCandidate.last_soprano = []
        startCandidate.last_cadence = "X"
        push!(startCandidate.last_soprano, previous_sequence[end][3][4][1])
        push!(startCandidate.last_four, previous_sequence[end][2])
        push!(activeset, startCandidate)
    end

    best_solution, estimator, voicingindeces, cost  = BB(function_potential, function_potential_unsorted, rhythm_map, data, voicing_permutations, 
                voicing_permutations_raw, valid_function_table, phase, activeset, previous_sequence)

    return best_solution, cost
end

prepare (generic function with 1 method)

In [16]:
function BB(function_potential, function_potential_unsorted, rhythm_map, data, voicing_permutations, 
        voicing_permutations_raw, valid_function_table, phase, activeset, previous_sequence)
        
    # Initialize global search variables
    best_solution = []
    best_score = 0
    total_depth = size(data, 2)
    println("Amount of timesteps to optimise: ", total_depth)
    
    ##### TEST ZONE new heuristic
    println("Building lookup-table list...")
    # Estimator: [t, ancient_chord][prev_chord, current_chord]
    estimator = []
    estimator, voicingindeces = initialiseBetterHeuristic(voicing_permutations_raw, total_depth)
    println("Lookup-table built.")
    #####
    
    # Populate active set with all permutations of the possible first functions
    
    score = []
    
    bestval_func = 499
    bestval_voices = 499
    println("Initial function score upper bound: ", bestval_func, ", voicing score upper bound: ", bestval_voices)
    solution_list = []

    # TEST ZONE: Manually set desired function at timestep [T]
    
    #function_potential[5] = [(0, "D64")]
    #function_potential[5] = [(0, "S65")]
    #function_potential_local[8] = [(0, "T")]
    #function_potential[12] = [(0, "T")]
    #function_potential[13] = [(0, "S")]
    #function_potential[14] = [(0, "D64")]
    #function_potential[15] = [(0, "D")]
    

    iterations = 0
    pruned = 0
    processed = 0
    solutions = 0
    last_solution = 0

    
    println("Starting search...")
    
    start = time()
    prev_depth = 1
    hit_bottom = false
    lowest_depth = total_depth
    best_solution = 0
    minlist = [0 for x in 1:total_depth+1]
    
    # Test best first
    wait_list = candidateSequence[]
    #
    done_list = Array{candidateSequence, 1}[[] for i=1:total_depth]
    dead_list = Array{candidateSequence, 1}[[] for i=1:total_depth]
    while !isempty(activeset)
        
        iterations += 1
        active_node = pop!(activeset)

        depth = size(active_node.sequence, 1)
        child_depth = depth+1
        
        if hit_bottom
           if depth < lowest_depth
                lowest_depth = depth
            end
        end
        # Before continuing, check if we need to update lookup table
        if depth < prev_depth 
            # Propagate all others
            
            for i in Iterators.reverse(depth:total_depth-1)
                if !isempty(done_list[i])
                    done_list[i] = sort!(done_list[i], by = c -> c.total_cost)
                end
            
                best = 0

                while !isempty(done_list[i])
                    candidate = popfirst!(done_list[i])

                    if best == 0
                        best = candidate
                        if minlist[i+1] > candidate.total_cost_per_timestep[end] - candidate.total_cost_per_timestep[i] || minlist[i+1] == 0
                            minlist[i+1] = candidate.total_cost_per_timestep[end] - candidate.total_cost_per_timestep[i]
                        end
                    end
                        
                    curr = voicingindeces[candidate.sequence[i][3]]
                    prev = curr
                    anc = curr
                    if i >= 2
                        prev = voicingindeces[candidate.sequence[i-1][3]]
                        if i >= 3
                            anc = voicingindeces[candidate.sequence[i-2][3]]
                        else
                            anc = prev
                        end
                    else
                        prev = curr
                    end
                    
                    cost_to_end = candidate.total_cost_per_timestep[end] - candidate.total_cost_per_timestep[i]
                    currentval = estimator[i, anc, prev, curr]
                    if currentval == -1 || (cost_to_end <= currentval && best.total_cost <= candidate.total_cost)

                        estimator[i, anc, prev, curr] = cost_to_end
                        if i > 1
                            push!(done_list[i-1], candidate)
                        end
                    else
                        continue
                    end  
                 end
            end
        end
        
        
        # Generate children of active_node and corresponding optimistic bounds
        #for (cost, func) in function_potential_local[depth+1]
        for (cost, func) in Iterators.reverse(function_potential[depth+1]) 
        
            #active_node.sequence = (cost, func, voicings)
            # Initialise new candidate without voicings just to check for function legality
            newCandidate = candidateSequence()
            newCandidate.sequence = active_node.sequence
            newCandidate.function_cost = active_node.function_cost
            newCandidate.function_cost_per_timestep = active_node.function_cost_per_timestep
            newCandidate.voicing_cost = active_node.voicing_cost
            newCandidate.total_cost_per_timestep = active_node.total_cost_per_timestep
            newCandidate.num_mediants = active_node.num_mediants
            newCandidate.same_note = active_node.same_note
            newCandidate.last_four = active_node.last_four
            newCandidate.last_soprano = active_node.last_soprano
            newCandidate.last_cadence = active_node.last_cadence
            
            temp_child = (cost, func, active_node.sequence[end][3])
            push!(newCandidate.sequence, temp_child)
            child_depth = size(newCandidate.sequence, 1)
                
            newCandidate.function_cost = function_cost(newCandidate, rhythm_map, valid_function_table) 
            newCandidate.function_cost_per_timestep[child_depth] = newCandidate.function_cost

            # Get voicing indeces for previous timesteps to use for heuristic lookup table
            # Prev = active_note depth, Ancient = prev - 1
            i_previous_voicing = voicingindeces[newCandidate.sequence[end-1][3]]
            
            if child_depth > 2
                i_anc_voicing = voicingindeces[newCandidate.sequence[end-2][3]]
            else
                i_anc_voicing = i_previous_voicing
            end
                            
            # If illegal, or if the chosen function can not possibly improve upon the best function chain found, prune it
            
            if newCandidate.function_cost > 999 
                pruned += 1                              
                # Mark all function voicings as invalid for the current combination as invalid

                if child_depth != total_depth
                    for v in voicing_permutations[func]
                        v_index = voicingindeces[v]
                        if estimator[child_depth, i_anc_voicing, i_previous_voicing, v_index] == -1
                            estimator[child_depth, i_anc_voicing, i_previous_voicing, v_index] = 1000
                        end
                    end
                end

                deleteat!(newCandidate.sequence, size(newCandidate.sequence, 1))
                continue
            else
            
                deleteat!(newCandidate.sequence, size(newCandidate.sequence, 1))
            end
            
            # Child : [(func1_cost, "func1_name", [func1_voicing_list], func1_total_func_cost, func1_total_voice_cost, func1_distance_cost), (function2_cost, "function2_name", , [function2_voicing_list]), ...]
            for voicing in voicing_permutations[func]
                
                #Create child with full voicing information
                child = deepcopy(newCandidate)
                temp_child = (cost, func, voicing)
                push!(child.sequence, temp_child)
                solution_found = false
                                        
                i_curr_voicing = voicingindeces[child.sequence[end][3]]
                # Calculate current voicing cost
                child.voicing_cost = voicing_cost(child, rhythm_map, data, false, false)
                  
                
                # Calculate child total cost (function + voice)
                child.total_cost = child.function_cost + child.voicing_cost
                child.total_cost_per_timestep[child_depth] = child.total_cost
                
                

                if child_depth == total_depth
                        child_heuristic = 0          
                    else
                        child_heuristic = estimator[child_depth, i_anc_voicing, i_previous_voicing, i_curr_voicing][1]                 
                end
                                        
                # Get heuristic. If -1, it has not been explored yet and should be replaced with x*remaining_timesteps (x = 0, 1, ..., adjustable!)                                                 
                if child_heuristic == -1
                    child_total_heuristic = 0*((total_depth-1) - child_depth)   #(15 * (total_depth-1) - child_depth # Inaccurate heuristic, but ensures a solution of a certain quality
                    child_estimated_cost = child.total_cost + child_total_heuristic
                else
                    child_estimated_cost = child.total_cost + child_heuristic
                    new_prop = true
                end 

                processed += 1
                if child_estimated_cost >= bestval_func + bestval_voices || child.total_cost + minlist[child_depth+1] > bestval_func + bestval_voices
                    

                    # Update estimator lookup table with cost of last-step + estimated cost if est is not -1
                    if child_depth != total_depth 
                        if estimator[child_depth, i_anc_voicing, i_previous_voicing, i_curr_voicing] == -1
                            estimator[child_depth, i_anc_voicing, i_previous_voicing, i_curr_voicing] = 1000
                        end 
                    end  
                                   
                    deleteat!(child.sequence, size(child.sequence, 1))
                    pruned += 1 
                
                elseif child_depth == total_depth
                    hit_bottom = true
                    solutions += 1
                    prev = voicingindeces[child.sequence[end-1][3]]
                    anc = voicingindeces[child.sequence[end-2][3]]
                    ancanc = voicingindeces[child.sequence[end-3][3]]
                                                    
                                                 
                    old = estimator[child_depth-1, ancanc, anc, prev]
                    new = child.total_cost_per_timestep[child_depth] - child.total_cost_per_timestep[child_depth-1]
                    if old == -1 || new < old
                        estimator[child_depth-1, ancanc, anc, prev] = 
                                                    (child.total_cost_per_timestep[child_depth] - child.total_cost_per_timestep[child_depth-1])
                        solution_found = true
                        currentbest = child
                    end
                                                
                    # Update estimator lookup table with cost of last-step only # KAN HENDE DEN MÅ PROPAGERE OGSÅ ETTER NODE SLUTT
                    
                    #println(estimator[child_depth, i_previous_voicing, :])
                            
                    if child.total_cost < bestval_func + bestval_voices
                                        
                        last_solution = iterations
                        currentbest = child
                        solution_found = true
                        bestval_func = child.function_cost
                        bestval_voices = child.voicing_cost 
                        best_solution = child
                        pushfirst!(solution_list, child)

                               
                        println("New solution")
                        println("function cost: ", child.function_cost)
                        println("child voicing cost: ", child.voicing_cost)
                        println("Best yet: ", bestval_func + bestval_voices)                                  
                    end
                else                                
                        
                    ### TEST
                    push!(wait_list, child)
                    #push!(activeset, child)
                end
                     
                #Can safely prop if active_node at penultimate step
                #if child_depth == total_depth
                if solution_found
                    push!(done_list[total_depth-1], currentbest)                                   
                end
                
            end
        end
        
        wait_list = sort!(wait_list, by = c -> c.total_cost)
                                            
        while !isempty(wait_list)
            push!(activeset, pop!(wait_list))                               
        end
        
        if iterations % 10000 == 0
            println("Iterations done: ", iterations)
            println("Nodes in active set: ", size(activeset, 1))
            println("Lowest depth: ", lowest_depth, " current depth: ", depth)
        end     
            
        if  iterations == 500000
            println("Timed out")
            #display(activeset)
            for timestep in done_list
                if !isempty(timestep)
                    for candidate in timestep
                        display(candidate)
                    end
                end
            end   
            display(minlist)
            break
        end
        prev_depth = depth     
    end
    
    elapsed = time() - start
    println("##: Done. Total checked: ", processed, ", Pruned: ", pruned, ", completed: ", solutions, ", time: ", elapsed, ", efficiency: ", (solutions)/elapsed)
    println("Results: bestval_func: ", bestval_func, ", bestval_voices: ", bestval_voices)
    return solution_list, estimator, voicingindeces, bestval_func+bestval_voices
end

#estimator = []
#solution_list, estimator, voicingindeces = prepare(data[:, 1:16], functions, function_potential, function_potential_unsorted)
    
#plant = 1
#println(plant)
                                       

BB (generic function with 1 method)

In [17]:
#Global Initialisations
mutable struct candidateSequence
    sequence::Array{Tuple{Int8,String,Array{Tuple{Int16,Int16},1}},1}
    function_cost::Int16
    voicing_cost::Int16
    total_cost::Int16
    total_cost_per_timestep::Array{Int16,1}
    function_cost_per_timestep::Array{Int16,1}
    num_mediants::Int8
    same_note::Array{Int8, 1}
    last_four::Array{String, 1}
    last_soprano::Array{Int8, 1}
    last_cadence::String
    candidateSequence() = new()
end

Tonics = ["T", "T/3", "T/5", "T-5"]
Tonics_dependent = ["T64", "T54"]
Subdominants = ["S", "S/3", "S/5", "S65", "S65/6", "S6"]
Dominants = ["D", "D/3", "D/5", "D-5", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7", "D64", "D54"]
Authentic_Dominants = ["D", "D7"]
Dominants_dependent = ["D64", "D54"]
Mediants = ["Ts", "Tm", "Ss", "Dm"]
beat_accentuation = Dict("O" => 1, "W" => 2, "S" => 3, "A" => 4, "C" => 4)
dominant_order = Dict("D" => 3, "D/3" => 3, "D/5" => 3, "D-5" => 3, "D64" => 1, "D54" => 2, "D7" => 5, "D7-1" => 4, "D7-5" => 5, "D7/3" => 5, "D7/5" => 5, "D7/7" => 5)
precursor_dominants = ["D", "D-5", "D64", "D54"]

4-element Array{String,1}:
 "D"  
 "D-5"
 "D64"
 "D54"

In [18]:
# time_signature = (beat, subdivision): (3,2), (4,2) supported

function optimiseMusic(filepath, time_signature)
    
    previous_sequence = []
    first_part = 0
    second_part = 0
    start = time()
    scale = 0
    estimator = 0
    voicingindeces = 0
    elapsed = 0
    total = 0
    key = 0
    total_cost = 0
    for i in 1:2
        # Retrieve and analyse distribution of input data
        phase = i
        if time_signature == (4,2)
            if phase == 1 
                startindex = 1
                finalindex = 16
            elseif phase == 2
                startindex = 16
                finalindex = 32
            end
        elseif time_signature == (3,2)
            if phase == 1 
                startindex = 1
                finalindex = 12
            elseif phase == 2
                startindex = 12
                finalindex = 24
            end 
        end            
                    
        full_data, data = initialiseData(filepath, startindex, finalindex)
        tonal_map = createTonalMap(full_data)
        println(tonal_map)
        key, scale = findScale(full_data, tonal_map)
        
        # Initialise function definitions and all voicing permutations
        functions = initialiseFunctionDefinitions(key, scale)
        function_potential, function_potential_unsorted = initialiseFunctionPotentialList(startindex, finalindex, tonal_map, functions)
        voicing_permutations, voicing_permutations_raw = createVoicingPermutations(functions)
        valid_function_table = createValidFunctionTable(scale)

        # Free estimatorcontainer from memory to avoid out of memory crash
        
        if phase == 1
            first_part, cost = prepare(data, time_signature, functions, function_potential, function_potential_unsorted, 
                voicing_permutations, voicing_permutations_raw, valid_function_table, phase, previous_sequence)
            push!(previous_sequence, first_part[1].sequence[end])
        else
            second_part, cost = prepare(data, time_signature, functions, function_potential, function_potential_unsorted, 
                voicing_permutations, voicing_permutations_raw, valid_function_table, phase, previous_sequence)
        end
                        
        total_cost += cost
        #display(valid_function_table)
        
    end
    elapsed = time() - start
    println("#Key: ", key, "Scale: ", scale)
    
    return first_part, second_part, scale, elapsed, total_cost  #first_part, second_part, scale, elapsed, estimator, voicingindeces
end
estimator = []

0-element Array{Any,1}

# # Collect solution voicings and create piano roll

In [20]:
using Glob

## Convert back to piano roll
function makePianoRoll(notes, filenumber, i, scale)
    pianoroll = zeros(Int8, 128, size(notes,1))
    counter = 1
    for (j, set) in enumerate(notes[:])
        for voice in set
            if scale == "Minor" && (j == size(notes,1) || j == size(notes,1)-1) && voice[2] == 3
                pianoroll[voice[1]+2, counter] = 1
            else
                pianoroll[voice[1]+1, counter] = 1
            end
        end
        counter = counter + 1
    end
    empty = zeros(Int8, 1, size(notes, 1))
    new = vcat(empty, pianoroll)

    savepath = "../../results/optimizer/pianoroll/$filenumber.csv"
    println(savepath)
    writedlm(savepath,  new, ',')
    println(" End of file ")
    println(" ")
end
    
#println(glob("*.csv", "../../results/neuralnet/pianoroll"))
for filepath in glob("*.csv", "../../results/neuralnet/pianoroll")   
    
    #filepath = "../../results/neuralnet/pianoroll/AC-11-Wann-nn.csv"
    println("Optimising: ", filepath)
    filenumberandextension = split(filepath, "-")[end]
    filenumber = split(filenumberandextension, ".")[1]

    first_part, second_part, scale, elapsed, totalcost = optimiseMusic(filepath, (4,2))

    path = split(filepath, "/")[end]
    stringlist = split(path, "-")[1:end]
    filename = stringlist[1] * "-" * stringlist[2] * "-" * stringlist[3] * "-opt"   
        
    solution_notes_list = []
    solution_functions_list = []
    solutionlist = []
    solution = deepcopy(first_part[1].sequence)

    solution = push!(solutionlist, solution)

    for timestep in second_part[1].sequence[2:end-1]
        push!(solutionlist[1], timestep)
    end

    push!(solutionlist[1], second_part[1].sequence[end-1])

    println("#Total time for optimisation of piece: ", elapsed)
    println("#Total cost of piece: ", totalcost)
    solution = solutionlist[1]

    amount = 10
    if size(solution, 1) < 10
        amount = size(solution, 1)
    end

    push!(solution_notes_list, [timestep[3] for timestep in solution])
    push!(solution_functions_list, [timestep[2] for timestep in solution])
    for line in solution
        display(line)
    end

    println(solution_functions_list[1])

    makePianoRoll(solution_notes_list[1], filename, 1, scale)

end

Optimising: ../../results/neuralnet/pianoroll/AC-06-Nun-nn.csv
[0 1 0 0 0 0 0 0 0 1 0 0; 0 0 2 0 0 0 0 0 0 0 0 1; 0 1 0 0 1 0 0 0 0 1 0 0; 0 0 2 0 0 0 2 0 0 0 0 0; 0 0 1 0 1 0 0 0 1 0 0 1; 0 0 0 0 0 0 2 0 0 2 0 0; 0 0 0 0 1 0 0 0 2 0 0 1; 0 0 0 0 2 0 0 0 0 2 0 0; 0 0 0 0 1 0 1 0 0 2 0 0; 0 0 0 0 1 0 0 0 1 2 0 0; 0 0 0 0 0 0 1 0 1 1 0 1; 0 0 0 1 0 0 0 0 1 1 0 1; 0 0 0 1 1 0 1 0 0 0 0 1; 0 0 0 1 1 0 1 0 1 0 0 0; 0 1 0 0 1 0 1 0 0 0 0 1; 0 0 0 0 1 0 3 0 0 0 0 0; 0 0 0 1 0 0 1 0 0 0 0 2; 0 0 0 0 1 0 0 0 2 0 0 1; 0 0 0 0 2 0 0 0 1 0 0 1; 0 0 0 0 1 0 0 0 0 1 0 2; 0 0 0 0 2 0 0 0 0 2 0 0; 0 0 1 0 1 0 0 0 0 1 0 1; 0 0 0 0 1 0 1 0 1 1 0 0; 0 0 1 0 0 0 1 0 0 1 0 1; 0 0 1 0 0 0 1 0 1 1 0 0; 0 0 0 0 1 0 0 0 1 1 0 1; 0 1 0 0 1 0 0 0 0 2 0 0; 0 1 0 0 1 0 0 0 0 2 0 0; 0 1 0 0 0 0 1 0 0 2 0 0; 0 0 1 0 0 0 2 0 0 1 0 0; 0 1 1 0 0 0 0 0 2 0 0 0; 0 1 0 0 0 0 1 0 0 2 0 0; 0 1 0 0 0 0 1 0 1 1 0 0]
Key Counter is [0 9 10 4 22 0 21 0 16 30 0 16]
Key Score is [78; 50; 108; 30; 118; 62; 66; 99; 29; 124; 44; 88]

(0, "T", Tuple{Int16,Int16}[(45, 1), (57, 1), (61, 3), (64, 5)])

(0, "Ss", Tuple{Int16,Int16}[(47, 1), (54, 5), (59, 1), (62, 3)])

(0, "T/3", Tuple{Int16,Int16}[(49, 3), (52, 5), (57, 1), (69, 1)])

(9, "S", Tuple{Int16,Int16}[(50, 1), (54, 3), (62, 1), (69, 5)])

...

(15, "D", Tuple{Int16,Int16}[(55, 1), (62, 5), (67, 1), (71, 3)])

(9, "D", Tuple{Int16,Int16}[(55, 1), (67, 1), (71, 3), (74, 5)])

(9, "T", Tuple{Int16,Int16}[(48, 1), (67, 5), (72, 1), (76, 3)])

(9, "T", Tuple{Int16,Int16}[(48, 1), (67, 5), (72, 1), (76, 3)])

##: Done. Total checked: 126479, Pruned: 137117, completed: 16, time: 2.57099986076355, efficiency: 6.223259769157759
Results: bestval_func: 174, bestval_voices: 69
#Key: 1Scale: Major
#Total time for optimisation of piece: 9.381999969482422
#Total cost of piece: 467
["T", "D7/7", "T/3", "S", "D7-1", "T", "S", "S", "T/3", "Tm", "S", "D7", "Ts", "S", "D", "D", "D7/7", "T/3", "S", "D", "D7/7", "T/3", "Ts", "Ss", "T/3", "S", "D7/7", "T/3", "D", "D", "T", "T"]
../../results/optimizer/pianoroll/SMP-72-chorl-opt.csv
 End of file 
 
